In [1]:
target_lang="en"  # change to your target lang

model_user = "Aanchan"
model_repo = "psst_model_cer_2"
model_name = f"{model_user}/{model_repo}"

from datasets import load_dataset

username = "jindaznb"  # change to your username

ngram_order = 3
repo_user = "jindaznb"
repo_name = f"europarl_bilingual_kenlm_{ngram_order}-gram_phoneme"
repo_path_remote = f"{repo_user}/{repo_name}"
repo_path_local = f"{repo_name}"

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
file_path = "dataset_phonemes.txt"

with open(file_path, "r") as file:
    lines = file.readlines()

num_lines = len(lines)
print("Number of lines in the file:", num_lines)


Number of lines in the file: 990161


In [3]:
from huggingface_hub import notebook_login

!git config --global user.email "jindaz.work@outlook.com"
!git config --global user.name "j"

notebook_login()

In [4]:
from huggingface_hub import create_repo, Repository

try:
  repo = Repository(repo_path_local, clone_from=repo_path_remote)
  print(f"\nRepository {repo_path_remote} already exists.")
except:
  create_repo(repo_path_remote, exist_ok=True)
  repo = Repository(repo_path_local, clone_from=repo_path_remote)
  print("\nRepository created.")

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/europarl_bilingual_kenlm_3-gram_phoneme is already a clone of https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram_phoneme. Make sure you pull the latest changes with `repo.git_pull()`.



Repository jindaznb/europarl_bilingual_kenlm_3-gram_phoneme already exists.


# train wordlevel ngram model

In [6]:
# !kenlm/build/bin/lmplz -o 5 <"dataset_phonemes.txt" > "5gram.arpa" --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/dataset_phonemes.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 104269206 types 48
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:576 2:15701515264 3:29440344064 4:47104544768 5:68694134784
Substituting fallback discounts for order 0: D1=0.5 D2=1 D3+=1.5
Statistics:
1 48 D1=0.5 D2=1 D3+=1.5
2 1438 D1=0.503546 D2=0.532523 D3+=0.0379641
3 27827 D1=0.454491 D2=0.987406 D3+=1.49656
4 274963 D1=0.519184 D2=1.04709 D3+=1.56705
5 1397737 D1=0.548146 D2=1.0195 D3+=1.4856
Memory estimate for binary LM:
type       kB
probing 31701 assuming -p 1.5
probing 33483 assuming -r models -p 1.5
trie     9648 without quantization
trie     3985 assuming -q 8 -b 8 quantization 
trie     9224 assuming -a 22 arr

In [5]:
!head -20 5gram.arpa

head: cannot open ‘5gram.arpa’ for reading: No such file or directory


In [6]:
# with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
#   has_added_eos = False
#   for line in read_file:
#     if not has_added_eos and "ngram 1=" in line:
#       count=line.strip().split("=")[-1]
#       write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
#     elif not has_added_eos and "<s>" in line:
#       write_file.write(line)
#       write_file.write(line.replace("<s>", "</s>"))
#       has_added_eos = True
#     else:
#       write_file.write(line)

In [9]:
# if ngram_order > 1:
#   !kenlm/build/bin/build_binary {repo_path_local}/5gram.arpa {repo_path_local}/5gram.bin

Reading europarl_bilingual_kenlm_3-gram_phoneme/5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [10]:
repo.push_to_hub(commit_message="Upload n-gram model")

Adding files tracked by Git LFS: ['5gram.arpa']. This may take a bit of time if the files are large.


Upload file 5gram.arpa:   0%|          | 1.00/40.9M [00:00<?, ?B/s]

Upload file 5gram.bin:   0%|          | 1.00/31.0M [00:00<?, ?B/s]

To https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram_phoneme
   9ce5853..3ae4dcb  main -> main



'https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram_phoneme/commit/3ae4dcbfe060b21b4591c2498e47ac739040e5b1'

In [13]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
vocab_dict

{'<pad>': 0,
 '<sil>': 41,
 '<spn>': 42,
 '<unk>': 43,
 'AA': 1,
 'AE': 2,
 'AH': 3,
 'AO': 4,
 'AW': 5,
 'AY': 6,
 'B': 7,
 'CH': 8,
 'D': 9,
 'DH': 10,
 'DX': 11,
 'EH': 12,
 'ER': 13,
 'EY': 14,
 'F': 15,
 'G': 16,
 'HH': 17,
 'IH': 18,
 'IY': 19,
 'JH': 20,
 'K': 21,
 'L': 22,
 'M': 23,
 'N': 24,
 'NG': 25,
 'OW': 26,
 'OY': 27,
 'P': 28,
 'R': 29,
 'S': 30,
 'SH': 31,
 'T': 32,
 'TH': 33,
 'UH': 34,
 'UW': 35,
 'V': 36,
 'W': 37,
 'Y': 38,
 'Z': 39,
 'ZH': 40,
 '<s>': 44,
 '</s>': 45}

In [16]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="europarl_bilingual_kenlm_3-gram_phoneme/5gram.arpa",
)

Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/europarl_bilingual_kenlm_3-gram_phoneme/5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Space token ' ' missing from vocabulary.
Unigrams and labels don't seem to agree.
Only 48 unigrams passed as vocabulary. Is this small or artificial data?


In [17]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

ValueError: The tokens {'F', 'V', 'EH', 'L', 'IY', 'AY', 'EY', 'DH', 'TH', 'CH', 'R', 'Z', 'AH', 'Y', 'P', 'SH', 'OY', 'OW', 'W', 'UW', 'B', 'G', 'UH', 'N', 'ZH', 'NG', 'HH', 'D', 'DX', 'AA', 'M', 'AE', 'AW', 'AO', 'IH', 'T', 'JH', 'K', 'ER', 'S'} are defined in the tokenizer's vocabulary, but not in the decoder's alphabet. Make sure to include {'F', 'V', 'EH', 'L', 'IY', 'AY', 'EY', 'DH', 'TH', 'CH', 'R', 'Z', 'AH', 'Y', 'P', 'SH', 'OY', 'OW', 'W', 'UW', 'B', 'G', 'UH', 'N', 'ZH', 'NG', 'HH', 'D', 'DX', 'AA', 'M', 'AE', 'AW', 'AO', 'IH', 'T', 'JH', 'K', 'ER', 'S'} in the decoder's alphabet.

In [10]:
from huggingface_hub import Repository

repo = Repository(local_dir="model_staging_phoneme", clone_from=model_name)

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Aanchan/psst_model_cer_2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/360M [00:00<?, ?B/s]

Download file training_args.bin:  97%|#########7| 3.34k/3.43k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##9       | 1.00k/3.43k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/360M [00:00<?, ?B/s]

In [11]:
processor_with_lm.save_pretrained("model_staging_phoneme")

NameError: name 'processor_with_lm' is not defined

In [2]:
import pandas as pd

df = pd.read_csv('torgo.csv')
df['audio'] = df['audio'].apply(lambda x: f"/work/van-speech-nlp/data/torgo/{x}")
df.to_csv('torgo.csv', index=False)